In [5]:
import os
import logging

import psycopg2
import psycopg2.extensions
from pymongo import MongoClient
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, Float, MetaData, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from subprocess import call

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Postgres
#call("bash /home/Egorov_load_project.sh", shell=True)
engine = create_engine('postgresql://postgres:@{}@postgres_host')

In [15]:
# Postgres pandas
accounts = pd.read_sql('select * from public.v_accounts ', engine)
logger.info("select view ")
print(accounts.head())
logger.info(" {}".format(accounts.dtypes))

2018-09-03 16:21:11,672 : INFO : select view 
2018-09-03 16:21:11,681 : INFO :  account_number      object
account_name        object
client_name         object
report_date         object
acc_value          float64
department_name     object
dtype: object


         account_number   account_name   client_name report_date  acc_value  \
0  42301000000000000001  Счет депозита   Иванов А.С.  2015-01-01   -10000.0   
1  42301000000000000002  Счет депозита  Смирнов А.С.  2015-01-01   -50000.0   
2  45201000000000000003  Счет депозита  Соболев А.С.  2015-01-01    -1000.0   
3  45201000000000000004   Ссудный счет   Иванов А.С.  2015-01-01   580000.0   
4  45201000000000000005   Ссудный счет  Смирнов А.С.  2015-01-01   400000.0   

  department_name  
0      Московский  
1  Северозападный  
2      Московский  
3      Московский  
4  Северозападный  


In [70]:
# Аналог 12 запроса выгрузка клиентов 
clients = accounts['client_name']
clients

0     Иванов А.С.
1    Смирнов А.С.
2    Соболев А.С.
3     Иванов А.С.
4    Смирнов А.С.
Name: client_name, dtype: object

In [42]:
# запрос 1 счетов у клиента запрос 5
account_count = accounts.groupby(['client_name']).agg({'acc_value':['count']})
account_count

,acc_value
,count
client_name,
Иванов А.С.,2
Смирнов А.С.,2
Соболев А.С.,1


In [131]:
# Количество счетов по клиенту у которого больше всех средств аналог SQL запроса 7
grouped = accounts.groupby(['client_name']).agg({'acc_value':['sum']}).head(1).filter(items = ['client_name'])
pd.merge(accounts, grouped, on = ['client_name', 'client_name'], how = 'inner').agg({'acc_value':['count']})
#accounts.loc[accounts.groupby(['client_name']).agg({'acc_value':['sum']}).head(1).filter(items = ['client_name']).idxmax()]

/opt/conda/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,acc_value
count,2


In [107]:
# Аналог запроса 
accounts.groupby(['client_name']).agg({'acc_value':['sum']})

,acc_value
,sum
client_name,
Иванов А.С.,570000.0
Смирнов А.С.,350000.0
Соболев А.С.,-1000.0


In [106]:
# вывод клиентов с положительным остатком аналог запроса №9
window_sums = accounts.assign(
cumsum =accounts.groupby(['client_name'])['acc_value']
.cumsum()
) .query('cumsum >0').sort_values(['client_name', 'cumsum'], ascending = [True, False]).filter(items = ['client_name', 'cumsum'])
print(window_sums)

    client_name    cumsum
3   Иванов А.С.  570000.0
4  Смирнов А.С.  350000.0


In [13]:
account_count.to_csv("account_count.csv", sep='\t', encoding='utf-8')

In [14]:
window_sums.apply(lambda x: x).to_csv("window_sums.csv", sep='\t', encoding='utf-8')
2

2